# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.29,94,96,10.96,SH,1698731512
1,1,whitehorse,60.7161,-135.0538,1.43,98,100,1.03,CA,1698731525
2,2,bilibino,68.0546,166.4372,-17.70,81,56,1.73,RU,1698731490
3,3,waitangi,-43.9535,-176.5597,13.23,89,100,5.81,NZ,1698731829
4,4,chonchi,-42.6128,-73.8092,1.73,97,16,0.58,CL,1698731621


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_fig= city_data_df.hvplot.points("Lng","Lat",geo=True,size="Humidity",color="City",alpha=0.4,
                                tiles="OSM",frame_width=700, frame_height=500)

# Display the map
map_fig

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
new_cities_df=city_data_df.loc[(city_data_df["Max Temp"]< 27)&(city_data_df["Max Temp"] > 21) \
                              & (city_data_df["Wind Speed"] < 4.5)\
                              &(city_data_df["Cloudiness"]== 0)]

# Drop any rows with null values
new_cities_df= new_cities_df.dropna()

# Display sample data
new_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,ciudad insurgentes,25.2625,-111.7753,22.42,64,0,2.06,MX,1698731845
163,163,clearwater,27.9659,-82.8001,23.29,84,0,3.60,US,1698731865
236,236,aswan,24.0934,32.9070,25.61,53,0,1.54,EG,1698731893
336,336,ondjiva,-17.0667,15.7333,24.38,13,0,2.88,AO,1698731647
338,338,tazacorte,28.6290,-17.9293,22.54,92,0,2.06,ES,1698731920


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=new_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
62,ciudad insurgentes,MX,25.2625,-111.7753,64,
163,clearwater,US,27.9659,-82.8001,84,
236,aswan,EG,24.0934,32.9070,53,
336,ondjiva,AO,-17.0667,15.7333,13,
338,tazacorte,ES,28.6290,-17.9293,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ciudad insurgentes - nearest hotel: Hotel Restaurant Californias
clearwater - nearest hotel: The Fort Harrison Hotel
aswan - nearest hotel: Yaseen hotel
ondjiva - nearest hotel: Águia Verde Hotel
tazacorte - nearest hotel: App Leyma


,City,Country,Lat,Lng,Humidity,Hotel Name
62,ciudad insurgentes,MX,25.2625,-111.7753,64,Hotel Restaurant Californias
163,clearwater,US,27.9659,-82.8001,84,The Fort Harrison Hotel
236,aswan,EG,24.0934,32.9070,53,Yaseen hotel
336,ondjiva,AO,-17.0667,15.7333,13,Águia Verde Hotel
338,tazacorte,ES,28.6290,-17.9293,92,App Leyma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_fig =hotel_df.hvplot.points("Lng","Lat",scale=1, geo=True,size="Humidity",color="City",alpha=0.4,tiles="OSM",frame_width=800,frame_height=500,hover_cols=["Hotel Name", "Country"])

# Display the map
map_fig

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)